# Residual Networks - Step by Step

- In recent years, neural networks have become much deeper, with state-of-the-art networks evolving from having just a few layers (e.g., AlexNet) to over a hundred layers. There are a lot of benefits of using a very deep network. However, it doesn't always help. 

- A huge barrier to training them is vanishing gradients: very deep networks often have a gradient signal that goes to zero quickly, thus making gradient descent prohibitively slow. More specifically, during gradient descent, as you backpropagate from the final layer back to the first one, you are multiplying by the weight matrix on each step, and thus the gradient can decrease exponentially quickly to zero (or, in rare cases, grow exponentially quickly and "explode," from gaining very large values).     
    
- During training, you might therefore see the magnitude (or norm) of the gradient for the shallower layers decrease to zero very rapidly as training proceeds, as shown below:

<center> <img src="images/vanishing_grad_kiank.png" width="50%" height="50%"> </center>
<caption> <center> <b> Figure 1 </b>: Vanishing gradient. The speed of learning decreases very rapidly for the shallower layers as the network trains </center> </caption>
<br>
This problem was solved by He et al. that introduced the Residual Networks in 2015.

## Packages

In [ ]:
import numpy as np
import tensorflow as tf

from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D
from tensorflow.keras.initializers import random_uniform, glorot_uniform
from tensorflow.keras.models import Model

<a name='1'></a>
## 1 - Building a Residual Network

In ResNets, a "shortcut" or a "skip connection" allows the model to skip layers:  

<center> <img src="images/skip_connection_kiank.png" width="50%" height="50%"> </center>
<caption> <center> <b> Figure 2 </b>: A ResNet block showing a skip-connection <br> </center> </caption>

- The image on the left shows the "main path" through the network. The skip connection adds a shortcut to this path, as we can see at the image on the right. By stacking these ResNet blocks on top of each other, you can form a very deep network. 

- The addition of the skip connection can also makes it very easy for the block to learn the identity function. This means that you can stack on additional ResNet blocks with little risk of harming training set performance.  
    - On that note, there is also some evidence that the ease of learning an identity function accounts for ResNets' remarkable performance even more than skip connections help with vanishing gradients.

Two main types of blocks are used in a ResNet, depending mainly on whether the input/output dimensions are or not the same: "identity block" and the "convolutional block".

<a name='1-1'></a>
### 1.1 - The Identity Block

The identity block is the standard block used in ResNets, and corresponds to the case where the input activation ($a^{[l]}$) has the same dimension as the output activation ($a^{[l+2]}$).

<center> <img src="images/idblock2_kiank.png" width="60%" height="60%"> </center>
<caption> <center> <b> Figure 3 </b>: Identity block. Skip connection "skips over" 2 layers. </center> </caption>
<br>
<center> <img src="images/idblock3_kiank.png" width="60%" height="60%"> </center>
<caption> <center> <b> Figure 4 </b>: Identity block. Skip connection "skips over" 3 layers. </center> </caption>
<br>
The upper path is the "shortcut path." The lower path is the "main path." In this diagram, notice the CONV2D and ReLU steps in each layer. To speed up training, a BatchNorm step has been added.

In [ ]:
def identity_block(X, f, n_f, training=True, initializer = random_uniform):
    """
    Implementation of the identity block from figure 4.
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    n_f -- python list of integers, defining the number of filters in the CONV layers of the main path
    training -- True: Behave in training mode
                False: Behave in inference mode
    initializer -- to set up the initial weights of a layer. Equals to random uniform initializer
    
    Returns:
    X -- output of the identity block, tensor of shape (m, n_H, n_W, n_C)
    """
    
    n_f1, n_f2, n_f3 = n_f
    
    ## Main path
    X_shortcut = X
    
    # Sequence 1
    X = Conv2D(filters = n_f1, 
               kernel_size = 1, 
               strides = (1,1), 
               padding = 'valid', 
               kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training = training)
    X = Activation('relu')(X)
    
    # Sequence 2
    X = Conv2D(filters = n_f2, 
               kernel_size = f, 
               strides = (1,1), 
               padding = 'same', 
               kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training = training)
    X = Activation('relu')(X) 

    # Sequence 3
    X = Conv2D(filters = n_f3, 
               kernel_size = 1, 
               strides = (1,1), 
               padding = 'valid', 
               kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training = training) 
    
    # Adding the input X
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)  

    return X

<a name='1-2'></a>
### 1.2 - The Convolutional Block

When the input and output dimensions don't match up, we use the convolutional block. To resize the input $x$ dimensions, we implement a CONV2D layer in the shortcut path. 

<center> <img src="images/convblock_kiank.png" width="60%" height="60%"> </center>
<caption> <center> <b>Figure 5 </b>: Convolutional block </center> </caption>
 
- The CONV2D layer on the shortcut path does not use any non-linear activation function. Its main role is to just apply a (learned) linear function to reduce the dimension of the input. 

In [ ]:
def convolutional_block(X, f, n_f, s=2, training=True, initializer = glorot_uniform):
    """
    Implementation of the convolutional block from figure 4
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    s -- Integer, specifying the stride to be used
    training -- True: Behave in training mode
                False: Behave in inference mode
    initializer -- to set up the initial weights of a layer. Equals to Glorot uniform initializer
    
    Returns:
    X -- output of the convolutional block, tensor of shape (m, n_H, n_W, n_C)
    """
    
    n_f1, n_f2, n_f3 = n_f
    
    ## Main path
    X_shortcut = X

    # Sequence 1
    X = Conv2D(filters = n_f1, 
               kernel_size = 1, 
               strides = (s,s), 
               padding='valid', 
               kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training = training)
    X = Activation('relu')(X)
    
    # Sequence 2
    X = Conv2D(filters = n_f2, 
               kernel_size = f, 
               strides = (1,1), 
               padding='same', 
               kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training = training) 
    X = Activation('relu')(X) 

    ## Sequence 3
    X = Conv2D(filters = n_f3, 
               kernel_size = 1, 
               strides = (1,1), 
               padding='valid', 
               kernel_initializer = initializer(seed=0))(X) 
    X = BatchNormalization(axis = 3)(X, training = training) 
    
    # Shortcut path 
    X_shortcut = Conv2D(filters = n_f3, 
                        kernel_size = 1, 
                        strides = (s,s), 
                        padding='valid', 
                        kernel_initializer = initializer(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3)(X_shortcut, training = training) 
    
    # Adding the main and shortcut paths output 
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    return X

<a name='2'></a>  
## 2 - Building a ResNet Model

Using the blocks implemented before, we will build the ResNet-50, as shown in the figure below.

<center> <img src="images/resnet_kiank.png" width="75%" height="75%"> </center>
<caption> <center> <b> Figure 5 </b>:  ResNet-50 model </center> </caption>    

In [ ]:
def ResNet50(input_shape, classes):
    """
    Stage-wise implementation of the architecture of the ResNet50.
    
    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """

    X_input = Input(input_shape)
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2), kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], s = 1)
    X = identity_block(X, 3, [64, 64, 256])
    X = identity_block(X, 3, [64, 64, 256])
    
    # Stage 3
    X = convolutional_block(X, f = 3, filters = [128, 128, 512], s = 2)
    X = identity_block(X, 3, [128, 128, 512])
    X = identity_block(X, 3, [128, 128, 512])
    X = identity_block(X, 3, [128, 128, 512])

    # Stage 4
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], s = 2)
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])

    # Stage 5
    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], s = 2)
    X = identity_block(X, 3, [512, 512, 2048])
    X = identity_block(X, 3, [512, 512, 2048])

    # Pooling layer
    X = AveragePooling2D(pool_size=(2,2))(X)

    # Output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', kernel_initializer = glorot_uniform(seed=0))(X)
    
    # Creating the model
    model = Model(inputs=X_input, outputs=X)

    return model